### **Typing in Julia**

Type is a tangible thing in julia and can be investigated via programming because of the meta nature of julia. 

* `<:` is a subtype of
* `A<:B` A is a subtype of B, viewed as boolean
* `Array{T}()` Instantiate with a type
* `Convert(A, b)` try Converting b to type A. 
* `subtypes()` get the subtypes of the current types
* `supertype()` get the super type of the current types
* `typeof()` get the Type object representing the type. 
* `isa(instance, type)` runtime check on the type. 
* `eltype()`: check the types indie of the container. 



In [10]:
# a function that prints types hiearchy
function SubTypeTree(t, level=1, indent=4)
    level == 1 && println(t)
    for s in subtypes(t)
      println(join(fill(" ", level * indent)) * string(s))
      SubTypeTree(s, level+1, indent)
    end
end
SubTypeTree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


In [26]:
# Type hirarchy 
display(BigInt <: Signed)
display(Rational <: Real)
display(Real <: Complex)
display(Complex<: Number)

true

true

false

true

When a container's inner type is not paramaterized. 

* Compare the inner type, if not matches then false, if matches, then let the results be determined by the type of the container. 
* any parametric type is a subtype of itself as a non-parametric type. 

In [19]:
# Think carefully about these examples. 
display(Vector <: Array)
display(Array{Complex} <: Array{Number})
display(Array{Complex} <: Array{Complex})
display(Vector{Complex} <: Array{Complex})
display(Vector{Complex} <: Vector)
display(Vector{Any} <: Array)

true

false

true

true

true

true

To make correct comparison involving the parametric types, we need to do this: 

In [28]:
display(Vector{Complex} <: Array{<:Number})
display(Vector{<:Complex} <: Array{<:Number})
display(Array{Int128} <: Array{<:Integer})

true

true

true

Union types are combining 2 types together. 

There are 2 special types I need to point out: 
* Nothing: This is just the null type. 
* Any: This is literally everything. 

WARNING: Only use `Union{T, Nothing}` for functions parameters! 
WARNING: Cannot use `Where` on struct definition. 

In [29]:
display(Integer<:Number<:Any)
display(Nothing <: Nothing)
function Nullable(arg::Union{Nothing, <:Number})
    if arg == nothing
        return "It's nothing."
    end
    return "It's something"
end
display(Nullable(3.4))
display(Nullable(nothing))

true

true

"It's something"

"It's nothing."

In [23]:
# Nothing is just itself on the hierarchy tree. 
display(supertype(Nothing))
display(SubTypeTree(Nothing))

Any

nothing

Nothing


Binded Parametric Type

* `A{T} <: B{T}` where `T` is the same, this is usually used for type and function definition 
* `where {T<:Type1, S <:Type1}` this is the syntax. 

In [30]:

# T is regarded as the same type for both parameters, this is convenient
function dotProduct(a::AbstractArray{T}, v::Vector{T})::AbstractArray{T} where {T<:Real}
"""
    Dot on the last dimension and then sum it up along the last dimension
"""
    

end
# where {T} is qually legit as where {T<:Any}. But, if T were to be Nullable, it needs toe be declared. 

dotProduct (generic function with 1 method)

Converting between different types.

Conversion is only possible if it's been implemented, usually you can go with the feels regardng whether 2 native types have conversion between them. 

**Potential Gotacha**

However, becareful, Julia will convert stuff automatically, it doesn't have type promotions, but it will convert implicitly under the following context: 

* Assigning to an array converts to the array's element type.
* Assigning to a field of an object converts to the declared type of the field.
* Constructing an object with new converts to the object's declared field types.
* Assigning to a variable with a declared type (e.g. local x::T) converts to that type.
* A function with a declared return type converts its return value to that type.
* Passing a value to ccall converts it to the corresponding argument type.

We can run type checking using the function `isa(instance, type)`




In [17]:
z = convert(Complex, 3.4)
display(isa(z, Number))

true

In [34]:
convert(Int64, 's')

115

* The type of a type of a number is a datatype 
* A Datatype is it's own type. 
* Union type cannot have super type. 
* Union Type is something on it's own, it doesn't join the hierarchy due to how it's paramaterized. 
* You can only `dump` on a union type. 

In [40]:
display(typeof(typeof(Number)))
display(typeof(typeof(Array)))
display(typeof(typeof(AbstractArray)))
display(typeof(typeof(DataType)))

DataType

DataType

DataType

DataType

In [41]:
supertype(Union{Number, Array})

LoadError: MethodError: no method matching supertype(::Type{Union{Number, Array}})
[0mClosest candidates are:
[0m  supertype([91m::DataType[39m) at operators.jl:43
[0m  supertype([91m::UnionAll[39m) at operators.jl:48

In [5]:
display(subtypes(Union))
# It's empty
methodswith(Union)
dump(Union{Number, Array, Matrix})


Type[]

Union{Number, Array}


In [34]:
display(eltype([1, 2, 3]))
display(eltype([1, 2, 3.0]))


Float64

**Type Alias** 

We can define a type that refers to some other types, Uniontypes as a global constant. 

Certain existing types are also Alias for other types. They are equivalent and just having a different names for clairty. 

In [20]:
const RationalNumber = Union{Integer, Rational}
dump(RationalNumber)

Union{Integer, Rational}


In [31]:
display(Vector)
display(Matrix)
A =  Array{Int64}(undef, 3, 3)
B = view(A, 1:2, 1:2)
display(typeof(B)) # I digressed, but subarray window does have a lot of types. 

Vector{T} where T (alias for Array{T, 1} where T)

Matrix{T} where T (alias for Array{T, 2} where T)

SubArray{Int64, 2, Matrix{Int64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}

### **Is an Instace of a Type defined**

